In [1]:
import numpy as np
from PIL import Image, ImageOps
from matplotlib import pyplot as plt
%matplotlib inline

## Import the original mnist dataset

In [2]:
from keras.datasets import mnist

ROWS = COLS = 28

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], ROWS, COLS, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], ROWS, COLS, 1).astype('float32')

Using TensorFlow backend.


## Remove nines

In [3]:
X_train = np.append(X_train, X_test, axis=0)
y_train = np.append(y_train, y_test, axis=0)

X_train = X_train[np.where(y_train!=9)]
y_train = y_train[np.where(y_train!=9)]

In [4]:
X_train.shape

(63042, 28, 28, 1)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=360, width_shift_range=0.1, height_shift_range=0.1, zoom_range=-0.3)

## Generate from every image mock images (zoomed, rotated, shifted...)

In [7]:
new_x_train = []
new_y_train = []
m = len(y_train)

gen = ImageDataGenerator(rotation_range=360, width_shift_range=0.1, height_shift_range=0.1, zoom_range=-0.3)

for i, (y, im) in enumerate(zip(y_train, X_train)):
    im[im<=30] = 0
    im[im>30] = 1
    im = im.reshape(1,28,28,1)
    it = gen.flow(im, batch_size=1)
    n = 10
    new_x_train.append(im)
    new_y_train += [y]*11
    for _ in range(10):
        im = it.next().astype('uint8')
        new_x_train.append(im)
    if (i+1)%100==0: print(f'Handled {i+1}/{m} rows. Generated {(i+1) * 11} images.', end='\r')

In [8]:
X_train = np.array(new_x_train).reshape(len(new_x_train),28,28,1)

In [9]:
y_train = np.array(new_y_train)

In [10]:
len(X_train)//len(np.unique(y_train))

77051

In [11]:
# add = 9
# divide = 10
# subtract = 11
# multiply = 12
# equals = 13

## Import the (few) instances of operators. Generate mock images (zoomed, rotated, shifted)

In [12]:
for symbol, alias in zip(['add', 'divide', 'subtract', 'multiply', 'equals'], [str(a) for a in range(9, 14)]):

    samples = ['pad_0.png', 'pad_1.png', 'pad_2.png', 'pad_3.png'] 
    n = 77000//len(samples)

    for s in samples:
        base_im = Image.open(f'data_gen/{symbol}/{s}')
        base_im = np.array(ImageOps.invert(base_im))
        base_im = base_im.reshape(1, ROWS, COLS, 1).astype('uint8')

        it = gen.flow(base_im, batch_size=1)
        arr = []
        for i in range(n):
            im = it.next().astype('uint8')
            im[im<=30] = 0
            im[im>30] = 1
            arr.append(im)
            if (i+1)%100==0: print(f'Generated {i+1}/{n} train images', end='\r')
        X_train = np.append(X_train, np.array(arr)[:,0,:], axis=0)
        y_train = np.append(y_train, [alias]*n, axis=0)
        print(f'Generated all train images for {s}!         ')

Generated all train images for pad_0.png!         
Generated all train images for pad_1.png!         
Generated all train images for pad_2.png!         
Generated all train images for pad_3.png!         
Generated all train images for pad_0.png!         
Generated all train images for pad_1.png!         
Generated all train images for pad_2.png!         
Generated all train images for pad_3.png!         
Generated all train images for pad_0.png!         
Generated all train images for pad_1.png!         
Generated all train images for pad_2.png!         
Generated all train images for pad_3.png!         
Generated all train images for pad_0.png!         
Generated all train images for pad_1.png!         
Generated all train images for pad_2.png!         
Generated all train images for pad_3.png!         
Generated all train images for pad_0.png!         
Generated all train images for pad_1.png!         
Generated all train images for pad_2.png!         
Generated all train images for 

In [13]:
len(X_train) - len(y_train)

0

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [15]:
batch_size = 128
num_classes = 14
epochs = 1

input_shape = (ROWS, COLS, 1)

## Shuffle the data, use 20% for validation and save several epochs

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [17]:
import keras

In [29]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [56]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_1_epoch')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 862769 samples, validate on 215693 samples
Epoch 1/1
862769/862769 [==============================] - 1599s 2ms/step - loss: 0.6211 - acc: 0.7975 - val_loss: 0.2696 - val_acc: 0.9188


In [57]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_2_epochs')

Train on 862769 samples, validate on 215693 samples
Epoch 1/1
862769/862769 [==============================] - 1644s 2ms/step - loss: 0.4031 - acc: 0.8734 - val_loss: 0.2348 - val_acc: 0.9283


In [58]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_3_epochs')

Train on 862769 samples, validate on 215693 samples
Epoch 1/1
862769/862769 [==============================] - 1583s 2ms/step - loss: 0.3723 - acc: 0.8865 - val_loss: 0.2068 - val_acc: 0.9377


In [59]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_4_epochs')

Train on 862769 samples, validate on 215693 samples
Epoch 1/1
862769/862769 [==============================] - 1844s 2ms/step - loss: 0.3571 - acc: 0.8914 - val_loss: 0.2162 - val_acc: 0.9374


In [60]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_5_epochs')

Train on 862769 samples, validate on 215693 samples
Epoch 1/1
862769/862769 [==============================] - 1476s 2ms/step - loss: 0.3460 - acc: 0.8957 - val_loss: 0.1947 - val_acc: 0.9413


In [18]:
mod = keras.models.load_model('models/mod_3_epochs')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Check out the misclassified test data

In [33]:
import pandas as pd

In [57]:
pred = mod.predict_classes(X_test)
mask = y_test.astype('uint8') != pred

In [75]:
(len(pred) - sum(mask)) / len(pred)

0.9375872188712661

In [80]:
misclf = pd.DataFrame()
indices = np.where(mask)

misclf['index'] = indices[0]
misclf['target'] = y_test[indices]
misclf['pred'] = pred[indices]

In [122]:
misclf.head(10)

,target,pred,index,tuple
0,7,1,16,"(7, 1)"
1,6,0,44,"(6, 0)"
2,6,5,73,"(6, 5)"
3,8,2,79,"(8, 2)"
4,1,7,86,"(1, 7)"
5,8,0,94,"(8, 0)"
6,6,0,100,"(6, 0)"
7,4,2,137,"(4, 2)"
8,2,7,158,"(2, 7)"
9,4,3,181,"(4, 3)"


In [97]:
from collections import Counter

In [105]:
c = Counter(misclf.target)
c

Counter({'7': 2010,
         '6': 1606,
         '8': 1775,
         '1': 1084,
         '4': 1391,
         '2': 1660,
         '5': 1265,
         '3': 1732,
         '0': 560,
         '11': 336,
         '12': 14,
         '10': 5,
         '9': 22,
         '13': 2})

In [ ]:
# add = 9
# divide = 10
# subtract = 11
# multiply = 12
# equals = 13

In [119]:
misclf['tuple'] = list(map(lambda l: (l[0], l[1]), misclf.values))

In [121]:
c = Counter(misclf.tuple)
c.most_common(10)

[(('7', 2), 871),
 (('2', 7), 807),
 (('1', 11), 608),
 (('6', 7), 587),
 (('8', 2), 485),
 (('3', 2), 414),
 (('5', 6), 409),
 (('7', 4), 404),
 (('2', 8), 403),
 (('4', 5), 395)]